In [1]:
import pandas as pd
import requests
from lxml.html import fromstring
from itertools import cycle
import traceback
import time
from bs4 import BeautifulSoup
import urllib.request
import random
from tqdm import tqdm

In [2]:
domain_sirens = pd.read_csv('/project/0_cleaning/output_cleaning/sirens_final.csv')
domains = domain_sirens['domain'].tolist()
domain_sirens.shape

(3767, 2)

In [3]:
def json_save(obj, name):
    json_content = json.dumps(obj)
    f = open(f"{name}.json","w")
    f.write(json_content)
    f.close()

In [80]:
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

proxies = get_proxies()
print(proxies)

{'51.77.210.229:3128'}


In [5]:
#proxy_pool = cycle(proxies)
domains_info = []

url_static = 'https://fr.trustpilot.com/review/'
for i, domain in tqdm(enumerate(domains)):
    
    # URL to request
    url = url_static + domain
    
    # Get a proxy from the pool
    #proxy = next(proxy_pool)
    
    try:
        response = requests.get(url, timeout=5)#,proxies={"http": proxy, "https": proxy})
        parsed_html = BeautifulSoup(response.content, "html.parser")
        
    except:
        general = ''
        email = ''
        telephone = ''
        agg_rating = ''
        industry = ''
    
    else:
        try:
            general = parsed_html.find(attrs={"data-initial-state": "business-unit-info"}).get_text().strip('\n')
            general = json.loads(general)
        except:
            general = ''

        try:
            email = parsed_html.find(attrs={"type": "application/ld+json"}).get_text().split('"email":')[1].split(",")[0].strip('"')
        except:
            email = ''

        try:
            telephone = parsed_html.find(attrs={"type": "application/ld+json"}).get_text().split('"telephone":')[1].split(",")[0].strip('"')
        except:
            telephone = ''

        try:
            agg_rating = parsed_html.find(attrs={"type": "application/ld+json"}).get_text().split('"aggregateRating":')[1].split("},")[0] + '}'
            agg_rating = json.loads(agg_rating)
        except:
            agg_rating = ''

        try:
            industry = parsed_html.find(attrs={"data-initial-state": "business-unit-tracking-properties"}).get_text()
            industry = json.loads(industry)
        except:
            industry = ''
            
    finally:
        domains_info.append({
            'domain': domain,
            'general': general,
            'email': email,
            'telephone': telephone,
            'agg_rating': agg_rating,
            'industry': industry
        })
        json_save(domains_info, 'trustpilot')
           
    time.sleep(2)


3767it [2:20:14,  2.18s/it]


In [7]:
len(domains_info)

3767